In [1]:

import pandas as pd
import numpy as np
import pickle as pkl
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np

import time
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sb
# import visual tools
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-talk')
plt.style.use('bmh')

#plt.rcParams['font.family'] = 'DejaVu Sans Mono'
#plt.rcParams['font.size'] = 9.5
plt.rcParams['font.weight'] = 'medium'
#plt.rcParams['figure.figsize'] = 10,7
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

# import util libs

from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 777
###
import pystan
from scipy.stats import pearsonr, zscore
from MFDFA import fgn
# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sb.set()
from tqdm import tqdm

import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import minmax_scale
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import Normalizer
# from sklearn.preprocessing import QuantileTransformer
# from sklearn.preprocessing import PowerTransformer

import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')

import stylised_facts_data_utilities.createLOB as createLOB

%matplotlib inline
import os
os.getcwd()


'/home/ak/Documents/Research/PaperCode/stylised_facts'

In [7]:
os.listdir('/media/ak/My Passport/')

['DBT_data (2).zip',
 'NK1_Index',
 '.Trash-1000',
 '11-1225-dr6-ecological-perspective-on-future-of-computer-trading(1).pdf',
 '11-1225-dr6-ecological-perspective-on-future-of-computer-trading.pdf',
 '5a388a9c_f2e6_4115_8140_0986c17d381d_604.pdf',
 'abd071102.pdf',
 'backup',
 'books',
 'CO1_Comdty',
 'CryptoExperiments',
 'DBT Data',
 'DBTLOBFiles',
 'DBT_data',
 'DBT_data (1).zip',
 'DBT_data (4)',
 'DBT_data (4).zip',
 'DBT_data (5)',
 'DBT_data (5).zip',
 'DBT_data.zip',
 'DBT_data_2.zip',
 'Does_Viterbi_work.ipynb',
 'gaussian process',
 'GC1_Comdty',
 'G_1_Comdty-20181028',
 'Install Discovery for Mac.dmg',
 'Install Discovery for Windows.exe',
 'jae.1158.pdf',
 'kernels (1).pdf',
 'Laptop Stuff',
 'lecture notes advanced topics',
 'OldDataForCleanUp',
 'PhD_Kylie-Anne Richards_v20180823.pdf',
 'plot_svm_regression.ipynb',
 'Python and Computational Books',
 'Python Books',
 'quant market material',
 'Reference Theses for Ph.D Thesis',
 'remarkable_refund.pdf',
 'RL materials',


In [8]:
folder= '/media/ak/My Passport1/ExperimentData/ActivityClockData/'
folderList = os.listdir(folder)
folderList
clocksDataList =[s for s in folderList if ('ClocksData') in s]


In [9]:
idxFile = 20
fileLoc = "".join((folder, clocksDataList[idxFile]))
symbolID =fileLoc.split("_")[1:3]
symbolText=("".join((symbolID[0]," ", symbolID[1])))

In [10]:
fileLoc

'/media/ak/My Passport1/ExperimentData/ActivityClockData/ClocksData_G_1_Comdty_20180504_.pkl'

In [11]:

pickle_to_file = pkl.load(open(fileLoc, "rb"))

In [12]:
keys = pickle_to_file.keys()

In [13]:
keys

dict_keys(['OriginalDF', 'TickBarDf', 'VolumeBarDf', 'DollarVolumeBarDf'])

In [14]:
originaldf=pickle_to_file[list(keys)[0]]
tickdf=pickle_to_file[list(keys)[1]]
volumedf = pickle_to_file[list(keys)[2]]
dollarvolumedf = pickle_to_file[list(keys)[3]]

In [15]:
dollarvolumedf.columns.values

array(['BidQuoteId', 'BidSize', 'QuoteTime', 'type_x', 'BestBid',
       'TradeTime', 'AskQuoteId', 'AskSize', 'time_y', 'type_y',
       'BestAsk', 'TradeId', 'TradeVolume', 'TradedTime', 'type',
       'TradePrice', 'TradeId', 'TimeStamp', 'milliSeconds',
       'DollarVolume', 'MicroPrice', 'TradeSize', 'DollarVolumeTraded'],
      dtype=object)

In [16]:

testY =np.array(volumedf.MicroPrice.pct_change().dropna())

In [17]:
def returnsPulse(x):
    cumsum = np.cumsum(x)
    y =  np.vstack([cumsum, np.triu( np.expand_dims( cumsum, axis=0 ) - np.outer( np.ones( len( cumsum )  ), cumsum ).transpose() ) ] ) 
    return y

In [18]:
# def returnsPulse(x):
#     cumsum = np.cumsum(x)
#     y = np.triu(np.vstack( [cumsum] + [cumsum - cumsum[i] for i in range(len(cumsum))]))
#     return pd.DataFrame(y)

In [19]:
y_df =pd.DataFrame(returnsPulse(testY))

In [20]:
def rollingSum(x):
    cumsum = np.cumsum( x )
    result = np.zeros( [len(x) + 1, len( x )])
    result[0, :] =cumsum
    for i in range( len( x )):
        result[ i+1, :] = (cumsum - cumsum[i])
    return np.triu( result )
        

    
def rollingMean(x):
    """
    Compute rolling mean of each bucket shifted by on time in each row
    """
    cumsum = np.cumsum( x )
    result = np.zeros( [len(x + 1), len( x )])
    den = np.arange(1, len( x ) + 1)
    result[0, :] =cumsum/den
    for i in range(len( x)):
        result[ i+1, :] = (cumsum - cumsum[i])/(den - (i +1))
    return np.triu( result)
    
 

In [21]:
def return_tstats_over_days( xs):
    """
    computes the t-statistic over multiple windows across many days
    
    """
    T, N = xs.shape
    result = np.zeros( [N+1, N])
    # compute the cumulative returns for trades with entry at tick at n=0
    cumsum = np.cumsum(x, axis =1) #of shape [T, N]
    
    result[0, :] = np.mean( cumsum, axis=0)/np.std( cumsum, axis=0)
    for n in range(N):
        ##subtract cumsum of shape [T,N] with cumsum[:,n] of [T,1], to compute the returns for trades with entry at n-th tick
        returns = cumsum - np.expand_dims(cumsum[:, n], axis =1)
        ##then compute the t-statistics over time(first) dimension
        result[n+1, :]= np.mean(returns, axis=0)/np.std(returns, axis=0)
    return np.triu(result)

In [22]:
x = np.random.normal(size =[10,20000])

In [23]:
t_stats= return_tstats_over_days(x)

In [24]:
pd.DataFrame(t_stats)

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,-0.406205,-0.116487,0.312885,0.455445,0.407694,0.384318,0.274210,0.424622,0.347749,0.474878,...,0.107620,0.107033,0.105554,0.107661,0.107932,0.110036,0.108787,0.106555,0.106435,0.106716
1,0.000000,0.168247,0.596086,0.705316,0.641638,0.772026,0.597828,0.865913,0.559248,0.712935,...,0.110330,0.109736,0.108259,0.110361,0.110630,0.112737,0.111485,0.109253,0.109123,0.109398
2,0.000000,0.000000,1.146201,1.157287,0.642745,0.423318,0.281715,0.412919,0.321619,0.434903,...,0.108370,0.107784,0.106314,0.108405,0.108676,0.110767,0.109522,0.107297,0.107175,0.107450
3,0.000000,0.000000,0.000000,0.371343,0.169544,0.067091,-0.020991,0.119072,0.102075,0.252335,...,0.103989,0.103418,0.101948,0.104043,0.104317,0.106400,0.105160,0.102940,0.102839,0.103122
4,0.000000,0.000000,0.000000,0.000000,-0.103031,-0.115170,-0.190537,-0.036499,-0.020935,0.143618,...,0.101087,0.100529,0.099066,0.101151,0.101427,0.103492,0.102261,0.100055,0.099967,0.100254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.256992,-0.386503,-0.221380,-0.152208
19997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.275947,-0.130063,-0.078585
19998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053418,0.087378
19999,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077430
